# Speed and Quality of Katz-Eigen Community Detection vs Louvain

### Main Take-aways
 Network (nodes, edges)  | Louvain Speed | KE Speed | Louvain Q/Qmax | KE Q/Qmax |
-----------------------  |:-------------:|:--------:|:--------------:|:---------:|
Amazon Product (328,679) | 0.472 s       | 0.089 s  | 0.882          | 0.968     |
AdHoc BA (1000,16975)    | 10.1 s        | 0.570 s  | 0.988          | 0.992     |
AdHoc BA (1000,5032)     | 13.7 s        | 0.276 s  | 0.308          | 0.956     |

** In this example, agglomerative clustering results in sub-par seperation of clusters in the Katz-eigen plot. It is shown that the louvain communities are actually representative of the distinct clusters in the katz-eigen plot. And by joining louvain communites together based on their placement in the katz-eigen plot, a much higher modularity (0.98) is achieved.

*** In this example, agglomerative clustering peforms very poorly, and as a result, results in poor communities. Again, by joining louvain communites together based on their placement in the katz-eigen plot, a much higher modularity (0.94) is achieved.

In [2]:
import zen
import pandas as pd
import numpy as np
from clusteringAlgo import lineClustering

#### Compare the speed of the Katz-eigen plot method of community detection with that of Louvain community detection, using the 328-node Amazon product network.

In [3]:
def katz(G,tol=0.01,max_iter=1000,alpha=0.001,beta=1):
    iteration = 0
    centrality = np.zeros(G.num_nodes)
    while iteration < max_iter:
        iteration += 1          # increment iteration count
        centrality_old = centrality.copy()

        for node in G.nodes_():
            Ax = 0
            for neighbor in G.neighbors_(node):
                weight = G.weight_(G.edge_idx_(neighbor,node))
                Ax += np.multiply(centrality[neighbor],weight)

                #Ax += centrality[neighbor]      #exclude weight due to overflow in multiplication

            centrality[node] = np.multiply(alpha,Ax)+beta

        if np.sum(np.abs(np.subtract(centrality,centrality_old))) < tol:
            return centrality

In [4]:
def modular_graph(Size1, Size2, edges1, edges2, common, katz_alpha=0.001):
    g1 = zen.generating.barabasi_albert(Size1,edges1)
    avgDeg1 = (2.0 * g1.num_edges)/g1.num_nodes
    lcc1 = np.mean(zen.algorithms.clustering.lcc_(g1))
    
    g2 = zen.generating.barabasi_albert(Size2,edges2)
    avgDeg2 = (2.0 * g2.num_edges)/g2.num_nodes
    lcc2 = np.mean(zen.algorithms.clustering.lcc_(g2))
    
    Size = Size1 + Size2
    G = zen.Graph()
    for i in range(Size):
        G.add_node(i)

    for edge in g1.edges_iter():
        u = edge[0]
        v = edge[1]
        G.add_edge(u,v)

    for edge in g2.edges_iter():
        u = edge[0]+Size1
        v = edge[1]+Size1
        G.add_edge(u,v)

    # Select random pairs of nodes to connect the subgraphs
    join_nodes = np.empty((common,2),dtype=np.int64)
    nodes1 = np.random.randint(0,Size1,size=common)
    nodes2 = np.random.randint(Size1,Size,size=common)
    join_nodes[:,0] = nodes1
    join_nodes[:,1] = nodes2

    for edge in join_nodes:
        if not G.has_edge(edge[0],edge[1]):
            G.add_edge(edge[0],edge[1])
    
    return G

In [5]:
def modularity(G,classDict,classList):
    Q = zen.algorithms.modularity(G,classDict)
    # Maximum Modularity
    count=0.0
    for e in G.edges():
        n1 = G.node_idx(e[0])
        n2 = G.node_idx(e[1])
        if classList[n1] == classList[n2]:
            count += 1
    same = count / G.num_edges
    rand = same - Q
    qmax = 1 - rand
    return Q, qmax

In [6]:
def MSE(x,y,m):
    '''
    Not mean-squared-error, just too lazy to chance the function name.
    Measures the orthogonal distance points are from the line through origin with slope m
    '''
    x_ = (m*y+x)/(m**2 + 1)
    y_ = (y*(m**2) + m*x)/(m**2 + 1)
    return np.sum(np.power(x-x_,2) + np.power(y-y_,2))
    

def slope(angle):
    '''
    Calculates linear slope of line with angle from x-axis (radians)
    '''
    return np.tan(angle)
    
def bound(x, y, slope, lineU, lineL):
    '''
    returns subset of x,y that are between the lines lineU and lineL
    '''
    upper = lineU(x,slope)
    lower = lineL(x,slope)
    return np.logical_and(y>=lower, y<=upper)

def get_offsets(m,d):
    '''
    calculates the shift of a line that has an orthogonal distance d
    from a line through the origin with slope m
    '''
    A = 1./(m**2) + 1
    B = -2*A
    C = (1./(m**2))+1 - d**2
    
    x1 = (-B + np.sqrt(B**2 - 4*A*C))/(2*A)
    
    y1 = m + (1./m) - (x1/m)
    
    b1 = y1 - m*x1
    return b1

def lineFinder(x, y, dtheta=0.01, dx=0.05):
    theta=0
    
    line = lambda x,m: m*x
    b_up = lambda x,m: m*(x)-get_offsets(m,dx)
    b_low= lambda x,m: m*(x)+get_offsets(m,dx)
    
    thetas = np.arange(1e-4,np.pi/2,dtheta)
    error = np.empty(len(thetas))
    
    for i,theta in enumerate(thetas):
        m = slope(theta)
        X = x[bound(x,y,m,b_up,b_low)]
        Y = y[bound(x,y,m,b_up,b_low)]
        error[i] = MSE(X,Y,m)*(len(X)/float(len(x)))
    
    return thetas, error

def optimizeAngles(angle,error,N,plot=False):
    N=10
    x = pd.Series(error).rolling(window=N).mean().iloc[N-1:].values
    x_ = np.diff(x)
    t = angle[N:]
    ang = []
    for i in range(len(x_)):
        if i != 0 and i != len(x_)-1:
            if x_[i-1] < 0 and x_[i] >= 0:
                ang.append(t[i])
    if plot:
        for a in ang:
            plt.vlines(ymin=np.min(x),ymax=np.max(x),x=a*180/np.pi,linestyle='--',color='grey')
        plt.plot(angle[N-1:]*180/np.pi,x)
        plt.xlabel('Angle (deg.)')
        plt.ylabel('Sum of Squared Orth. Error')
        plt.title('Identification of Local Minima')
        plt.show()

    return ang

def getGroups(x,y,angles):
    if len(angles) == 1:
        m = slope(angles[0])
        y_ = x*m
        g1 = np.where(y<=y_)[0]
        g2 = np.where(y>y_)[0]
        return [g1,g2]
    elif len(angles) == 2:
        m1 = slope(angles[0])
        m2 = slope(angles[1])
        y1_ = x*m1
        y2_ = x*m2
        g1 = np.where(y<=y1_)[0]
        g2 = np.where(np.logical_and(y>y1_,y<=y2_))[0]
        g3 = np.where(y>y2_)[0]
        return [g1,g2,g3]
    
def lineClustering(x,y, dtheta=0.01, dx=0.05, window=10, plot=False):
    angles, errors = lineFinder(x,y,dtheta=dtheta,dx=dx)
    best_angles = optimizeAngles(angles,errors,window,plot=plot)
    clusters = getGroups(x,y,best_angles)
    return clusters

In [7]:
def ke_community_detection(G,dtheta=0.01,dx=0.5,window=10):
    evc = zen.algorithms.eigenvector_centrality_(G)
    kc = katz(G,alpha=1e-4)
    
    #scale
    evc = evc - np.min(evc)
    evc = evc / np.max(evc)
    kc  = kc - np.min(kc)
    kc = kc / np.max(kc)
    
    clusters = lineClustering(evc,kc,dtheta=dtheta,dx=dx,window=window)
    
    ClassDict = {}
    ClassList = np.zeros(G.num_nodes)
    for i,c in enumerate(clusters):
        ClassDict[i] = [G.node_object(x) for x in c]
        ClassList[c]=i

    q,qmax = modularity(G,ClassDict,ClassList)
    print '%d communities found.'%(i+1)
    print 'Q:            %.3f'%q
    print 'Normalized Q: %.3f'%(q/qmax)

In [8]:
from zen.algorithms.community import louvain
def louvain_community_detection(G):
    cset = louvain(G)

    comm_dict = {}
    comm_list = np.zeros(G.num_nodes)
    for i,community in enumerate(cset.communities()):
        comm_dict[i] = community.nodes()
        comm_list[community.nodes_()] = i

    q,qmax = modularity(G,comm_dict,comm_list)
    print '%d communities found.'%(i+1)
    print 'Q:            %.3f'%q
    print 'Normalized Q: %.3f'%(q/qmax)

### Test on Amazon Product Graph

In [9]:
G = zen.io.gml.read('amazon_product.gml',weight_fxn=lambda x: x['weight'])

In [10]:
%%time
ke_community_detection(G)

3 communities found.
Q:            0.359
Normalized Q: 0.769
CPU times: user 33.4 ms, sys: 1.63 ms, total: 35 ms
Wall time: 31.5 ms


In [11]:
%%time
louvain_community_detection(G)

14 communities found.
Q:            0.801
Normalized Q: 0.882
CPU times: user 370 ms, sys: 795 µs, total: 371 ms
Wall time: 371 ms


## Test on synthetic graphs

In [12]:
G_synth = modular_graph(500,500,15,20,100,katz_alpha=1e-4)
print "Nodes: %d"%G_synth.num_nodes
print "Edges: %d"%G_synth.num_edges

Nodes: 1000
Edges: 16975


In [13]:
%%time
ke_community_detection(G_synth)

3 communities found.
Q:            0.480
Normalized Q: 0.964
CPU times: user 329 ms, sys: 31.2 ms, total: 360 ms
Wall time: 329 ms


In [14]:
%%time
louvain_community_detection(G_synth)

2 communities found.
Q:            0.485
Normalized Q: 0.988
CPU times: user 11.8 s, sys: 0 ns, total: 11.8 s
Wall time: 11.8 s


In [15]:
G_synth = modular_graph(500,500,2,8,100,katz_alpha=1e-4)
print "Nodes: %d"%G_synth.num_nodes
print "Edges: %d"%G_synth.num_edges

Nodes: 1000
Edges: 5032


In [16]:
%%time
ke_community_detection(G_synth)

3 communities found.
Q:            0.099
Normalized Q: 0.190
CPU times: user 122 ms, sys: 5.3 ms, total: 127 ms
Wall time: 115 ms


In [17]:
%%time
louvain_community_detection(G_synth)

15 communities found.
Q:            0.273
Normalized Q: 0.304
CPU times: user 21.2 s, sys: 0 ns, total: 21.2 s
Wall time: 21.2 s


In [18]:
G_synth = modular_graph(1000,1000,4,7,100,katz_alpha=1e-4)
print "Nodes: %d"%G_synth.num_nodes
print "Edges: %d"%G_synth.num_edges

Nodes: 2000
Edges: 11035


In [19]:
%%time
ke_community_detection(G_synth)

2 communities found.
Q:            0.228
Normalized Q: 0.484
CPU times: user 228 ms, sys: 14.8 ms, total: 243 ms
Wall time: 228 ms


In [20]:
%%time
louvain_community_detection(G_synth)

17 communities found.
Q:            0.291
Normalized Q: 0.313
CPU times: user 2min 2s, sys: 1.14 ms, total: 2min 2s
Wall time: 2min 2s
